In [1]:
!pip install pyspark

In [2]:
# Solo se necesitan si se corre con spark-submit (o si se corre en Google Colab)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ejemplo3').getOrCreate()

In [3]:
df = spark.read.csv('info_ventas.csv', inferSchema=True, header=True)
df.show()

+--------+----------+------+
|Compania|   Persona|Ventas|
+--------+----------+------+
|    GOOG|       Sam| 200.0|
|    GOOG|    Carlos| 120.0|
|    GOOG|     Frank| 340.0|
|    MSFT|    Teresa| 600.0|
|    MSFT|       Amy| 124.0|
|    MSFT|   Vanessa| 243.0|
|      FB|     Carla| 870.0|
|      FB|      Sara| 350.0|
|    APPL|      Juan| 250.0|
|    APPL|     Linda| 130.0|
|    APPL|    Miguel| 750.0|
|    APPL| Christian| 350.0|
+--------+----------+------+



In [4]:
df.printSchema()

root
 |-- Compania: string (nullable = true)
 |-- Persona: string (nullable = true)
 |-- Ventas: double (nullable = true)



## Funciones

Más información en: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions

In [5]:
# Algunas funciones
from pyspark.sql.functions import avg, stddev, countDistinct

In [6]:
# Contar elementos distintos
df.select(countDistinct("Ventas")).show()

+----------------------+
|count(DISTINCT Ventas)|
+----------------------+
|                    11|
+----------------------+



In [7]:
# Desviación estándar de las ventas
df.select(stddev("Ventas")).show()

+-------------------+
|stddev_samp(Ventas)|
+-------------------+
| 250.08742410799007|
+-------------------+



In [8]:
# Promedio de las ventas
df.select(avg('Ventas')).show()
df.select(avg('Ventas').alias('Media de Ventas')).show()

+-----------------+
|      avg(Ventas)|
+-----------------+
|360.5833333333333|
+-----------------+

+-----------------+
|  Media de Ventas|
+-----------------+
|360.5833333333333|
+-----------------+



In [9]:
# Para controlar el número de dígitos 
from pyspark.sql.functions import format_number

df2 = df.select(avg('Ventas').alias('Media de ventas'))
df2.select(format_number('Media de ventas',3)).show()
df2.select(format_number('Media de ventas',3).alias('Media de ventas')).show()

+---------------------------------+
|format_number(Media de ventas, 3)|
+---------------------------------+
|                          360.583|
+---------------------------------+

+---------------+
|Media de ventas|
+---------------+
|        360.583|
+---------------+



## OrderBy

In [10]:
# Orden ascendente según una columna
df.orderBy("Ventas").show()

+--------+----------+------+
|Compania|   Persona|Ventas|
+--------+----------+------+
|    GOOG|    Carlos| 120.0|
|    MSFT|       Amy| 124.0|
|    APPL|     Linda| 130.0|
|    GOOG|       Sam| 200.0|
|    MSFT|   Vanessa| 243.0|
|    APPL|      Juan| 250.0|
|    GOOG|     Frank| 340.0|
|      FB|      Sara| 350.0|
|    APPL| Christian| 350.0|
|    MSFT|    Teresa| 600.0|
|    APPL|    Miguel| 750.0|
|      FB|     Carla| 870.0|
+--------+----------+------+



In [11]:
# Orden descendente según una columna
df.orderBy(df["Ventas"].desc()).show()

+--------+----------+------+
|Compania|   Persona|Ventas|
+--------+----------+------+
|      FB|     Carla| 870.0|
|    APPL|    Miguel| 750.0|
|    MSFT|    Teresa| 600.0|
|      FB|      Sara| 350.0|
|    APPL| Christian| 350.0|
|    GOOG|     Frank| 340.0|
|    APPL|      Juan| 250.0|
|    MSFT|   Vanessa| 243.0|
|    GOOG|       Sam| 200.0|
|    APPL|     Linda| 130.0|
|    MSFT|       Amy| 124.0|
|    GOOG|    Carlos| 120.0|
+--------+----------+------+



## Agrupamiento: GroupBy

In [12]:
# Agrupamiento de datos
df.groupBy("Compania")

In [13]:
# Métodos sobre datos agrupados: media
df.groupBy("Compania").mean().show()

+--------+-----------------+
|Compania|      avg(Ventas)|
+--------+-----------------+
|    APPL|            370.0|
|    GOOG|            220.0|
|      FB|            610.0|
|    MSFT|322.3333333333333|
+--------+-----------------+



In [14]:
# Otros métodos sobre datos agrupados

#df.groupBy("Company").max().show()
#df.groupBy("Company").min().show()
#df.groupBy("Company").count().show()
#df.groupBy("Company").sum().show()

In [15]:
# Agregación usando columnas
df.agg({'Ventas':'sum'}).show()
df.agg({'Ventas':'max'}).show()

+-----------+
|sum(Ventas)|
+-----------+
|     4327.0|
+-----------+

+-----------+
|max(Ventas)|
+-----------+
|      870.0|
+-----------+



In [ ]:
v = df.agg({'Ventas':'max'}).collect()
v = v[0].asDict()
v['max(Ventas)']

In [16]:
# Agregación sobre las columnas de los datos agrupados

df2 = df.groupBy("Compania")
df2.agg({"Ventas":'max'}).show()

+--------+-----------+
|Compania|max(Ventas)|
+--------+-----------+
|    APPL|      750.0|
|    GOOG|      340.0|
|      FB|      870.0|
|    MSFT|      600.0|
+--------+-----------+



### Ejemplo: Agrupamiento con fechas

Se desea el volumen promedio por cada año

In [17]:
df = spark.read.csv('/content/appl_stock.csv', inferSchema=True, header=True)
df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows



In [19]:
from pyspark.sql.functions import year

# Creación de una columna "Year"
df = df.withColumn("Year", year(df['Date']))
df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+----+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+----------+----------+------------------+------------------+---------+------------------+----+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
+----------+----------+----------+------------------+------------------+---------+------------------+----+
only showing top 5 rows



In [20]:
# Media (de todas las columnas) agrupada por año
df.groupBy('Year').mean().show()

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|         avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|  5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|          1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523|   2014.0|
|2012|     576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002|       1.319642044E8| 74.81383696800002|   2012.0|
|2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60

In [21]:
df.select(['Year', 'Volume']).groupBy('Year').mean().show()

+----+---------+--------------------+
|Year|avg(Year)|         avg(Volume)|
+----+---------+--------------------+
|2015|   2015.0|  5.18378869047619E7|
|2013|   2013.0|          1.016087E8|
|2014|   2014.0| 6.315273055555555E7|
|2012|   2012.0|       1.319642044E8|
|2016|   2016.0|  3.84153623015873E7|
|2010|   2010.0|1.4982631666666666E8|
|2011|   2011.0|1.2307474166666667E8|
+----+---------+--------------------+



In [22]:
# Media usando solo las columnas "Year" y "Volume"
df2 = df.groupBy('Year').mean().select(['Year', 'Avg(Volume)'])
df2.show()

+----+--------------------+
|Year|         Avg(Volume)|
+----+--------------------+
|2015|  5.18378869047619E7|
|2013|          1.016087E8|
|2014| 6.315273055555555E7|
|2012|       1.319642044E8|
|2016|  3.84153623015873E7|
|2010|1.4982631666666666E8|
|2011|1.2307474166666667E8|
+----+--------------------+



In [23]:
# Cambio de formato
df2.select('Year',format_number('Avg(Volume)',2) ).show()

+----+-----------------------------+
|Year|format_number(Avg(Volume), 2)|
+----+-----------------------------+
|2015|                51,837,886.90|
|2013|               101,608,700.00|
|2014|                63,152,730.56|
|2012|               131,964,204.40|
|2016|                38,415,362.30|
|2010|               149,826,316.67|
|2011|               123,074,741.67|
+----+-----------------------------+



In [24]:
df2.select('Year',format_number('Avg(Volume)',2).alias("Promedio Volumen")).show()

+----+----------------+
|Year|Promedio Volumen|
+----+----------------+
|2015|   51,837,886.90|
|2013|  101,608,700.00|
|2014|   63,152,730.56|
|2012|  131,964,204.40|
|2016|   38,415,362.30|
|2010|  149,826,316.67|
|2011|  123,074,741.67|
+----+----------------+

